In [ ]:
### Import Stuff ###

import numpy as np

import mne
from mne.forward import make_forward_dipole
from mne.simulation import simulate_evoked

from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from nilearn.plotting import plot_anat
from nilearn.datasets import load_mni152_template

import math
from mpl_toolkits import mplot3d


from mne.transforms import apply_trans
from os import path as op

import random

import scipy.signal as ss
import scipy.stats as st


from scipy.fft import rfft, rfftfreq, irfft


### Load Data ###

EEG_h = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_h.npy',allow_pickle=True)
EEG_m = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_processed/EEG_m.npy',allow_pickle=True)

P = np.load('/home/e/etayhay/frankm/Mazza2023/data/simulations_raw/Healthy/DIPOLEMOMENT_Seed1000.npy')
moments = P['HL23PN1'][80000:][:400]

### Define functions ###

def roi_to_surface_frank(roi_xyzs,vtx_xyzs,faces):
    faces_xyz = np.mean(vtx_xyzs[faces],axis=1)
    face_nearestroi_inds = cKDTree(faces_xyz).query(roi_xyzs, k=1)[1]
    vtx_nearestroi_inds = cKDTree(vtx_xyzs).query(roi_xyzs, k=1)[1]
    
    return vtx_nearestroi_inds,face_nearestroi_inds

### Setup Source Space ###
# the raw file containing the channel location + types

data_path = '/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6/'
raw_fname = data_path + 'sample_audvis_raw.fif'
subjects_dir = data_path + 'subjects'
subject = 'sample'

# The transformation file obtained by coregistration
trans = data_path + 'sample_audvis_raw-trans.fif'
info = mne.io.read_info(raw_fname)

# source space is just a mesh of grid points of possible dipole origins
src = mne.setup_source_space(subject,
                             spacing='oct4', # increase to oct6, 8, etc for more realistic output
                             add_dist=True,
                             subjects_dir=subjects_dir,verbose=False)

### Make BEM ###

print('\n BEM model: ')
model = mne.make_bem_model(subject='sample', ico=4,
                           conductivity=(0.3, 0.006, 0.3),
                           subjects_dir=subjects_dir,verbose=False)

print('\n BEM Solution: ')
fname_bem = mne.make_bem_solution(model,verbose=False)



fname_trans = data_path + 'sample_audvis_raw-trans.fif'
trans = mne.read_trans(fname_trans)




# Create Dummy Evoked to hold dipole data 
fname_ave = data_path + 'sample_audvis-ave.fif'
evoked = mne.read_evokeds(fname_ave, condition='Right Auditory',baseline=(None, 0),verbose=False)
evoked.pick_types(meg=False, eeg=True)

evoked.info['sfreq'] = 400
evoked.info['lowpass'] = 200

In [ ]:
dip_locs = [[-0.07074824+.0131,0.05937758,0.0837867],
            [-0.05889541+.0131,0.06404063-.0044,0.10122117-.01],
            [-0.0378412+.0131,0.06372997-.017,0.11839715],
            [-0.0506296+.0131,0.08445242-.007,  0.08830147],
            [-0.04332023+.0131,  0.02930477-.014,  0.13122222-.002]]

In [ ]:
# magnitude was too large before i think reduced dipole by 1e3

EEGs = [[],[]]

paths = ['/home/e/etayhay/frankm/Mazza2022_output/Healthy/DIPOLEMOMENT_Seed',
         '/home/e/etayhay/frankm/Mazza2022_output/SST/Circuit_output_4/DIPOLEMOMENT_Seed']

for cond in range(2):
    for s in range(60):
        eeg = []
        
        seeds = [i for i in range(1000,1060)]
        seeds.remove(1000+s) # don't want dlpfc activity copied elsewhere
        
        for c,coord in enumerate(dip_locs):
            # create dipole with timeseries from random sim
            seed = random.randrange(1000,1060)
            path = paths[0]+str(seed)+'.npy'
#             print(seed)

            if c==1: #ie dlpfc - [0]EEG9 F5, [1]EEG10 F3, [2]EEG11 F1, [3]EEG05 AF3, [4]EEG020 FC1
                if cond==0:
                    seed = 1000+s
                if cond==1:
                    seed = 2000+s
#                 print(cond,s,seed)
                path = paths[cond]+str(seed)+'.npy'
            
            print(cond,s,c,seed)
            P = np.load(path)
                
            moments = P['HL23PN1'][80000:][::400]

            coord = np.array([coord])

            vtx_inds,face_inds = roi_to_surface_frank(coord,src[0]['rr'],src[0]['tris'])

            ori = src[0]['nn'][vtx_inds]
            ori = ori.reshape(1,3)

            dip = mne.Dipole(times = np.arange(2000, 30001, 10), # timepoint for each momment
                             pos = np.full((len(moments),3),coord), # origin of each moment in head coordinates, stays same for all (fixed)
                             ori = np.full((len(moments),3),ori), # orientation of each dipole
                             amplitude = np.asarray([math.sqrt(((i[0]*0.000000000001)**2)+((i[1]*0.000000000001)**2)+((i[2]*0.000000000001)**2)) for i in moments]),
                             gof = np.full((len(moments)),100)) # goodness of fit, doesn't matter here


            fwd, stc = make_forward_dipole(dip, fname_bem, evoked.info, fname_trans,verbose=False)

            pred_evoked = simulate_evoked(fwd, stc, evoked.info, cov=None, nave=np.inf,verbose=False)

            eeg.append(pred_evoked.get_data()) # dipole eeg
            if seed in seeds:
                seeds.remove(seed) # remove this seed from possible choices so no duplicated activity, if statement for if dlpfc since already taken out
        
        
        eeg = np.array(eeg)
        eeg_summed = eeg.sum(axis=0) # add all eegs together
        EEGs[cond].append(eeg_summed)
        
EEGs = np.array(EEGs)        
np.save('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6EEGs_final_noscaling.npy',EEGs)

In [ ]:
EEGs = np.load('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6/EEGs_final_noscaling.npy')

fig,axes=plt.subplots(figsize=(15,5), ncols=3)

ts = np.arange(0,28.01,.01)
signal = np.array(EEGs[0][0][0])
sigmean = np.mean(signal)

fft = rfft(signal)
freq = rfftfreq(signal.size, d=1/100) # timestep is 1/sampling rate

dc = 1/len(signal)*np.abs(fft)[0]

axes[0].plot(ts,signal)
axes[0].axhline(sigmean,color='k',label='original sig mean')
axes[0].axhline(dc,color='yellow',ls='--',label='fft dc')

axes[1].plot(freq[:50],1/len(signal)*np.abs(fft)[:50])

axes[2].plot(ts[:len(irfft(fft))], irfft(fft))

axes[2].axhline(sigmean,color='k',label='original sig mean')
axes[2].axhline(dc,color='yellow',ls='--',label='fft dc')
axes[2].axhline(np.mean(irfft(fft)),color='green',ls=':',label='reconstructed sig mean',zorder=0)


axes[0].set_xlim(0,3)
axes[2].set_xlim(0,3)

_ = [ax.ticklabel_format(style='plain') for ax in axes]
axes[2].set_ylim(axes[0].get_ylim())

axes[0].set_title('Raw signal')
axes[0].set_ylabel('Potential (V)')
axes[0].set_xlabel('Time (s)')

axes[1].set_title('FFT')
axes[1].set_ylabel('Power')
axes[1].set_xlabel('Frequency (Hz)')

axes[2].set_title('Reconstructed signal from FFT')
axes[2].set_ylabel('Potential (V)')
axes[2].set_xlabel('Time (s)')


axes[0].legend(frameon=False,loc='upper right')
axes[2].legend(frameon=False,loc='upper right')

plt.tight_layout()


#################


# find mean dlpfc healthy

sigs = []
sigmeans = []

dcs = []

theta_alpha = []

dc_ta_ratio = []

for s in range(5):
    signal = EEGs[0][s][9] #EEG 010 index
    
    # get signal and signal mean(DC)
    sigs.append(signal)
    sigmeans.append(np.mean(signal))
    
    # FFT
    fft = rfft(signal)
    freqs = rfftfreq(signal.size, d=1/100) # timestep is 1/sampling rate
    
    # get DC
    dc = 1/len(signal)*np.abs(fft)[0]
    dcs.append(dc)
    
    #Mean theta alpha power
    l = int(4/freq[1]) #3.999
    h = int(12/freq[1])
    
    theta_alpha.append(np.mean(1/len(signal)*np.abs(fft[l:h])))
    
    dc_ta_ratio.append(theta_alpha[s]/dc)
    
fig,axes=plt.subplots(figsize=(15,5), ncols=4)

ts = np.arange(0,28.01,.01)

for s in range(5):
    ax = axes[0].plot(ts,sigs[s],lw=1)
    axes[1].scatter(0,dcs[s],color=ax[-1].get_color()) #get color of last plotted line so they match
    axes[1].scatter(1,sigmeans[s],color=ax[-1].get_color())
    axes[2].scatter(0,theta_alpha[s], color = ax[-1].get_color())
    axes[3].scatter(0,dc_ta_ratio[s], color = ax[-1].get_color())


axes[0].axhline(np.mean(dcs),color='k')
axes[1].scatter(.3,np.mean(dcs),color='k')

axes[1].scatter(.7, np.mean(sigmeans),color='k')
axes[2].scatter(.5, np.mean(theta_alpha), color = 'k')
axes[3].scatter(.5, np.mean(dc_ta_ratio), color = 'k')


axes[1].set_xticks([0,.3,.7,1])
axes[1].set_xticklabels(['DC','DC mean','Sigmean mean','sigmean'])

axes[0].set_title("Healthy DLPFC signals")
axes[0].set_ylabel('Potential')
axes[0].set_xlabel("Time (s)")

axes[1].set_title("Signal DCs")

axes[2].set_title('Theta Alpha Power')

axes[3].set_title('Theta Alpha / DC ratio')

plt.tight_layout()

ref_dc = np.mean(dcs)
ref_ta = np.mean(theta_alpha)

print('Healthy DLPFC DC:',ref_dc)
print('Healthy DLPFC TA:',ref_ta)



###################

fig, axes= plt.subplots(figsize=(15,5),ncols=4, sharey=True)

signal = EEGs[0][0][8][:2800]

fft = rfft(signal)

axes[0].plot(signal,label='F7')

dc = 1/len(signal)*np.abs(fft)[0]
dc_scale = dc/ref_dc

    
ta_scaling_factor = np.mean(1/len(signal)*np.abs(fft[l:h]))/ref_ta

newsig = signal - dc
newsig_scaled = newsig/ta_scaling_factor
newsig_scaled_shifted = newsig_scaled + dc/dc_scale

print('TA scale:', ta_scaling_factor)
print('DC scale:', dc_scale)

axes[1].plot(newsig)
axes[2].plot(newsig_scaled)
axes[3].plot(newsig_scaled_shifted)

axes[0].set_title('Original Signal')
axes[1].set_title('DC Removed')
axes[2].set_title('Amplitude Scaled')
axes[3].set_title('Shifted to DLPFC DC')

axes[0].plot(EEGs[0][0][9][:2800], color = 'k', label='F5 (dlPFC)')
axes[0].legend()


################


import scipy.signal as ss
scaled_eeg = np.zeros(shape=(2,60,59,2800))

dcs = []

h_ffts = []
h_wel = []

m_ffts = []
m_wel = []

fig,axes = plt.subplots(figsize=(15,5), ncols=4)
                 
for c in range(2):
    for s in range(60):
        for e in range(59): # all electrodes, will change important ones later
            
            dcs.append(1/len(signal)*np.abs(rfft(signal))[0])
            
            signal = EEGs[c][s][e][:2800]
            scaled_eeg[c][s][e] = signal
            
            if e in [8,9,10,4,19]: # important electrodes
                
                fft = rfft(signal)
                freq = rfftfreq(signal.size, d=1/100)
                
                dc = 1/len(signal)*np.abs(fft)[0]
                dc_scale = dc/ref_dc

                ta_scaling_factor = np.mean(1/len(signal)*np.abs(fft[l:h]))/ref_ta

                newsig = signal - dc
                newsig_scaled = newsig/ta_scaling_factor
                newsig_scaled_shifted = newsig_scaled + dc/dc_scale
                
                scaled_eeg[c][s][e] = newsig_scaled_shifted
                
                # welch method
                f_wel, Pxx_wel =ss.welch(x = signal,
                                        fs = 100,
                                        nperseg = 300,
                                        noverlap=100)
                
                                # welch method
                f_wel, Pxx_wel_scaled =ss.welch(x = newsig_scaled_shifted,
                                        fs = 100,
                                        nperseg = 300,
                                        noverlap=100)
                
                
                if s==0 and c==0:
                    axes[0].plot(freq[1:800],1/len(signal)*np.abs(fft)[1:800], alpha=.5)
                    axes[1].plot(freq[1:800],1/len(signal)*np.abs(rfft(newsig_scaled_shifted))[1:800], alpha=.5)
                    axes[2].plot(f_wel, Pxx_wel,alpha=.5)
                    axes[3].plot(f_wel, Pxx_wel_scaled,alpha=.5)
                    
                    if e==9:
                        axes[0].plot(freq[1:800],1/len(signal)*np.abs(fft)[1:800], alpha=1, color = 'k',zorder=np.inf)
                        axes[1].plot(freq[1:800],1/len(signal)*np.abs(rfft(newsig_scaled_shifted))[1:800], alpha=1, color = 'k',zorder=np.inf)
                        axes[2].plot(f_wel, Pxx_wel,alpha=.5, color = 'k',zorder=np.inf)
                        axes[3].plot(f_wel, Pxx_wel_scaled,alpha=.5, color = 'k',zorder=np.inf)
                        
                if c==0 and e==9:
                    h_ffts.append(1/len(signal)*np.abs(fft)[:800])
                    h_wel.append(Pxx_wel_scaled)
                    
                if c==1 and e==9: # if depression dlpfc, keep original
                    scaled_eeg[c][s][e] = EEGs[c][s][e][:2800]
                    
                    m_ffts.append(1/len(signal)*np.abs(fft)[:800])
                    m_wel.append(Pxx_wel)
                    
                    if s==0:
                        axes[0].plot(freq[1:800],1/len(signal)*np.abs(fft)[1:800], alpha=1, color='red',zorder=np.inf)
                        axes[1].plot(freq[1:800],1/len(signal)*np.abs(fft)[1:800], alpha=1, color='red',zorder=np.inf)
                        axes[2].plot(f_wel,Pxx_wel, color='red',zorder=np.inf)
                        axes[3].plot(f_wel,Pxx_wel, color='red',zorder=np.inf)


print('All electrode DC > 0:',np.all(np.array(dcs) > 0))

[ax.set_xlim(0,30) for ax in axes]
axes[0].set_title('Raw FFT')
axes[1].set_title('Scaled FFT')
axes[2].set_title('Raw Welch')
axes[3].set_title('Scaled Welch')



######################

h_ffts = np.array(h_ffts)
h_wel = np.array(h_wel)

m_ffts = np.array(m_ffts)
m_wel = np.array(m_wel)

fig, axes = plt.subplots(figsize=(15,5), ncols = 2)

axes[0].plot(freq[1:800], h_ffts.mean(axis=0)[1:], color = 'k')
axes[0].fill_between(freq[1:800],
                     h_ffts.mean(axis=0)[1:]+h_ffts.std(axis=0)[1:],
                     h_ffts.mean(axis=0)[1:]-h_ffts.std(axis=0)[1:], color = 'k', alpha = .5)

axes[0].plot(freq[1:800], m_ffts.mean(axis=0)[1:], color = 'red')
axes[0].fill_between(freq[1:800],
                     m_ffts.mean(axis=0)[1:]+m_ffts.std(axis=0)[1:],
                     m_ffts.mean(axis=0)[1:]-m_ffts.std(axis=0)[1:], color = 'red', alpha = .5)


axes[1].plot(f_wel,h_wel.mean(axis=0), color = 'k')
axes[1].fill_between(f_wel,
                    h_wel.mean(axis=0)+h_wel.std(axis=0),
                    h_wel.mean(axis=0)-h_wel.std(axis=0), color = 'k', alpha = .5)
axes[1].plot(f_wel,m_wel.mean(axis=0), color = 'red')
axes[1].fill_between(f_wel,
                    m_wel.mean(axis=0)+m_wel.std(axis=0),
                    m_wel.mean(axis=0)-m_wel.std(axis=0), color = 'red', alpha = .5)

[ax.set_xlim(0,25) for ax in axes]

In [ ]:
from numpy import std, mean, sqrt

def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)


In [ ]:
reference_freqs,_ = ss.welch(scaled_eeg[0][0][0],fs=100,nperseg=100*3,noverlap=100) #get frea

freqs = [[1,4],[4,8],[8,12],[12,16]]
freqlabels = ['delta','theta','alpha','beta']

colors = ['k', 'r']
for freq,freqlabel in list(zip(freqs,freqlabels)):
    psds = [[[[] for _ in range(len(scaled_eeg[0][0]))] for _ in range(len(scaled_eeg[0]))] for _ in range(2)]
    

    s = np.where(reference_freqs==freq[0])[0][0]
    e = np.where(reference_freqs==freq[1])[0][0]
    
    print(freq, freqlabel, s, reference_freqs[s], e, reference_freqs[e])
    
#     fig = plt.figure(figsize=(6,6))
#     ax = plt.subplot(111)
    for c in range(2):
        for r in range(60):
            for ch in range(len(scaled_eeg[c][r])):
                f,psd = ss.welch(scaled_eeg[c][r][ch],fs=100,nperseg=100*3,noverlap=100)
                psds[c][r][ch] = [np.mean(psd[s:e])]
#                 if ch == 9:
#                     ax.plot(psd,color = colors[c])

    # difference between depression - healthy
    psds = np.array(psds)
    diff = psds[1].mean(axis=0) - psds[0].mean(axis=0)
    diff = diff.reshape((59,1))
    diffhead = mne.EvokedArray(data=diff,info=evoked.info)
    
    #significance
    sigs = []
    ts = []
    
    
    for ch in range(59):
        t,p = st.ttest_ind(psds[1,:,ch].ravel(),psds[0,:,ch].ravel())
        sigs.append([p]) # p for signif, t gives direction (-ve for mdd > health)
        ts.append([t])
        
        if ch in [8,9,10,4,19]:
            if p<0.05:
                print(ch, p,cohen_d(psds[1,:,ch].ravel(),psds[0,:,ch].ravel()))
            

        
    sigs = np.array(sigs)
    sigplot = mne.EvokedArray(sigs,info=evoked.info)
    
    pmask = sigplot.data < 0.05
    mask_params = dict(markersize=10, markerfacecolor='yellow')

    
    # make mask of at least 5% power FC and if significant
    noisemask = psds[0].mean(axis=0) >= psds[0].mean(axis=0)[9]
    
    
    #### MAKE SURE MASK IS GOOD ###
    
    # make final mask
    finalmask = []
    
    for i in range(len(noisemask)):
        if noisemask[i][0] == True and pmask[i][0] ==True:
            finalmask.append([True])
        else:
            finalmask.append([False])

    finalmask = np.array(finalmask)        
    
    #plot difference and overlay significance
    diffhead.plot_topomap(times = [0],mask=finalmask, mask_params=mask_params,title=freqlabel + ' difference', contours =np.arange(-3,3,.5));

# Prep Images for Figure 6

## A

In [ ]:

roi_loc = np.array([[-0.05889541+.0131,0.06404063-.0044,0.10122117-.01]])

mri_pos = mne.head_to_mni(pos = roi_loc,
               subject=subject,
               mri_head_t=trans,
               subjects_dir=subjects_dir)

mri_pos


mni_pos = mne.head_to_mni(roi_loc, mri_head_t=trans,
                          subject=subject, subjects_dir=subjects_dir)

mri_pos = mne.head_to_mri(roi_loc, mri_head_t=trans,
                          subject=subject, subjects_dir=subjects_dir)

t1_fname = op.join(subjects_dir, subject, 'mri', 'T1.mgz')
fig_T1 = plot_anat(t1_fname, cut_coords=mri_pos[0], title='Dipole loc.')

template = load_mni152_template()
fig_template = plot_anat(template, cut_coords=mni_pos[0],
                         title='Dipole loc. (MNI Space)')

vtx_inds,face_inds = roi_to_surface_frank(roi_loc,src[0]['rr'],src[0]['tris'])

ori = src[0]['nn'][vtx_inds]
ori = ori.reshape(1,3)



dip = mne.Dipole(
    times = np.array([0]),
    pos = roi_loc,
    ori = ori,
    amplitude = np.array((moments[0,2]*0.000000001)**2).reshape(1,),
    gof=np.array([100]).reshape(1,),verbose=True)



fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111,projection='3d')
ax.set_yticklabels('')
ax.set_xticklabels('')
ax.set_zticklabels('')

dip.plot_locations(fname_trans,
               'sample',
               subjects_dir,mode='orthoview',ax=ax)

ax.set_ylabel('')
ax.set_xlabel('')
ax.set_zlabel('')


fig.savefig('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6/Figure6_A.jpg', facecolor='white', edgecolor='none',bbox_inches = "tight",dpi=1000)

## B

In [ ]:
roi_loc = np.array([[-0.05889541+.0131,0.06404063-.0044,0.10122117-.01]])

mri_pos = mne.head_to_mni(pos = roi_loc,
               subject=subject,
               mri_head_t=trans,
               subjects_dir=subjects_dir)

vtx_inds,face_inds = roi_to_surface_frank(roi_loc,src[0]['rr'],src[0]['tris'])

ori = src[0]['nn'][vtx_inds]
ori = ori.reshape(1,3)

dip = mne.Dipole(
    times = np.array([0]),
    pos = roi_loc,
    ori = ori,
    amplitude = np.array((moments[0,2]*0.000000001)**2).reshape(1,),
    gof=np.array([100]).reshape(1,),verbose=True)


print('\n Forward Model: ')
fwd, stc = make_forward_dipole(dip, fname_bem, evoked.info, fname_trans,verbose=False)

print('\n Predicting Activity:')
pred_evoked = simulate_evoked(fwd, stc, evoked.info, cov=None, nave=np.inf,verbose=False)

In [ ]:
# 10e-6 size of normal ERP which makes sense because we ahve 10e-6 less neurons

min(pred_evoked.data/10e-6), max(pred_evoked.data/10e-6)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(121)
ax2 = plt.subplot(122)

# create mask to show source locations
electrodes = [9]
emask = []
for i in range(60):
    if i not in electrodes:
        emask.append([False])
    else:
        emask.append([True])
        
emask = np.array(emask)
mask_params = dict(markersize=10, markerfacecolor='white')


pred_evoked.plot_topomap(times = [0],
                         axes=[ax,ax2],
                         contours = np.arange(-.01,0.01,.001),
                         title='',
                         mask = emask,
                         mask_params=mask_params);

fig.savefig('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6/Figure6_B.jpg', facecolor='white', edgecolor='none',bbox_inches = "tight",dpi=1000)

## C

In [ ]:
# finding good timepoints

plotting_eeg = []

for ch in scaled_eeg[1][0]:
    ch2 = np.array(ch)
    ch2 = ch - np.mean(ch)
    plotting_eeg.append(ch2)
    
plotting_eeg = np.array(plotting_eeg)

example_shift = mne.EvokedArray(data=plotting_eeg, info=evoked.info)


# find good timepoint that shows different sources best
data = example_shift.get_data(tmin=1.801,tmax=1.803)

# bring out dlpfc for easier view
data[8] = min(data)
data[9] = max(data)+1e-8

C = mne.EvokedArray(data=data, info=evoked.info)

# create mask to show source locations
electrodes = [4,8,9,10,19]
emask = []
for i in range(60):
    if i not in electrodes:
        emask.append([False])
    else:
        emask.append([True])
        
emask = np.array(emask)
mask_params = dict(markersize=10, markerfacecolor='white')

fig = plt.figure(figsize=(10,10))
ax = plt.subplot(121)
ax2 = plt.subplot(122)

C.plot_topomap(times = [0],
               axes=[ax,ax2],
               contours = np.arange(-.5,.5,.05),
               title='',
               vmin=-.5,
               vmax=.5,
               nrows=1,
              mask = emask,
              mask_params=mask_params);

fig.savefig('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6/Figure6_C.jpg', facecolor='white', edgecolor='none',bbox_inches = "tight",dpi=1000)

## D E F

In [ ]:
reference_freqs,_ = ss.welch(scaled_eeg[0][0][0],fs=100,nperseg=100*3,noverlap=100) #get frea

freqs = [[4,8],[8,12],[12,16]]
freqlabels = ['theta','alpha','beta']

colors = ['k', 'r']
#bounds = []
for freq,freqlabel in list(zip(freqs,freqlabels)):
    psds = [[[[] for _ in range(len(scaled_eeg[0][0]))] for _ in range(len(scaled_eeg[0]))] for _ in range(2)]
    

    s = np.where(reference_freqs==freq[0])[0][0]
    e = np.where(reference_freqs==freq[1])[0][0]
    
    print(freq, freqlabel, s, reference_freqs[s], e, reference_freqs[e])
    
    for c in range(2):
        for r in range(60):
            for ch in range(len(scaled_eeg[c][r])):
                f,psd = ss.welch(scaled_eeg[c][r][ch],fs=100,nperseg=100*3,noverlap=100)
                psds[c][r][ch] = [np.mean(psd[s:e])]

    # difference between depression - healthy
    psds = np.array(psds)
    
    diff = psds[1].mean(axis=0) - psds[0].mean(axis=0)
    diff = diff.reshape((59,1))
    diffhead = mne.EvokedArray(data=diff,info=evoked.info)
    
    #significance
    sigs = []
    ts = []
    

    for ch in range(59):
        t,p = st.ttest_ind(psds[1,:,ch].ravel(),psds[0,:,ch].ravel())
        sigs.append([p]) # p for signif, t gives direction (-ve for mdd > health)
        ts.append([t])
        
    sigs = np.array(sigs)
    sigplot = mne.EvokedArray(sigs,info=evoked.info)
    
    pmask = sigplot.data < 0.05
    mask_params = dict(markersize=10, markerfacecolor='yellow')
        
    # make mask of at least 40% power FC and if significant
    noisemask = psds[0].mean(axis=0) >= psds[1].mean(axis=0)[9]*(1/math.e)
    
    # make final mask
    finalmask = []
    
    for i in range(len(noisemask)):
        if noisemask[i][0] == True and pmask[i][0] ==True:
            finalmask.append([True])
        else:
            finalmask.append([False])

    finalmask = np.array(finalmask)        
    
    
    fig = plt.figure(figsize=(10,10))
    ax = plt.subplot(121)
    ax2 = plt.subplot(122)


    
    diffhead.plot_topomap(times = [0],
                          axes=[ax,ax2],
                          mask=finalmask,
                          mask_params=mask_params,
                          title='',
                          contours = np.arange(-0.000000004,0.000000004,0.0000000004),
                         vmin = -0.000000004,
                          vmax = 0.000000004);
    # find max and min for bounds first
    #bounds.append([min(diff),max(diff)])
    fig.savefig('/home/e/etayhay/frankm/Mazza2023/data/figures/Figure6/Figure6_'+str(freqlabel)+'.jpg', facecolor='white', edgecolor='none',bbox_inches = "tight",dpi=1000)